# Table 3.

Mean global, day-side, and night-side surface temperature ($K$) in the control and sensitivity simulations of Trappist-1e and Proxima b.

[Skip code and jump to the table](#Show-the-table)

----------------------------------

Import the necessary libraries.

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
from aeolus.calc import last_year_mean, spatial
from aeolus.core import Run

In [4]:
from commons import (
    DAYSIDE,
    NIGHTSIDE,
    PLANET_ALIASES,
    RUN_ALIASES,
)
import mypaths

## Load data

Create a dictionary of `Run` objects with preprocessed data.

In [5]:
runs = {}
for planet in PLANET_ALIASES.keys():
    for run_key in RUN_ALIASES.keys():
        label = f"{planet}_{run_key}"

        fname = mypaths.sadir / label / "_processed" / f"{label}.nc"

        runs[label] = Run(
            files=fname,
            name=label,
            planet=planet,
            processed=True,
        )

## Calculate mean temperatures

In [6]:
diags = {
    "$\overline{T_s}$": lambda cl: spatial(
        cl.extract_strict("surface_temperature"), "mean"
    ),
    "$\overline{T_{s,d}}$": lambda cl: spatial(
        cl.extract(DAYSIDE.constraint).extract_strict("surface_temperature"), "mean"
    ),
    "$\overline{T_{s,n}}$": lambda cl: spatial(
        cl.extract(NIGHTSIDE.constraint).extract_strict("surface_temperature"), "mean"
    ),
    "$T_{s,max}$": lambda cl: spatial(cl.extract_strict("surface_temperature"), "max"),
    "$T_{s,min}$": lambda cl: spatial(cl.extract_strict("surface_temperature"), "min"),
}

Loop over planets and runs and store results in a `DataFrame`.

In [7]:
results = {}
for planet in PLANET_ALIASES.keys():
    dfs = {}
    for run_key in RUN_ALIASES.keys():
        label = f"{planet}_{run_key}"

        _data = {}
        for vrbl, _calc in diags.items():
            _data[vrbl] = float(last_year_mean(_calc(runs[label].proc)).data)
        dfs[run_key] = pd.DataFrame(
            _data, index=pd.Index(name="Simulation", data=[run_key])
        )
    results[planet] = pd.concat(dfs, axis="index").droplevel(0).T
df = pd.concat(results).T
df = df.rename(index=RUN_ALIASES).rename(columns=PLANET_ALIASES)

Print the `DataFrame` to LaTeX.

In [8]:
df.to_latex(
    buf=mypaths.tabdir / "mean_sfc_temp_table.tex",
    formatters={k: lambda x: f"{x:.1f}" for k in df.columns},
    column_format="l" + "c" * len(diags) * len(PLANET_ALIASES),
    escape=False,
)

And show it here.

In [9]:
print(df.to_latex(
    formatters={k: lambda x: f"{x:.1f}" for k in df.columns},
    column_format="l" + "c" * len(diags) * len(PLANET_ALIASES),
    escape=False,
))

\begin{tabular}{lcccccccccc}
\toprule
{} & \multicolumn{5}{l}{Trappist-1e} & \multicolumn{5}{l}{Proxima b} \\
{} & $\overline{T_s}$ & $\overline{T_{s,d}}$ & $\overline{T_{s,n}}$ & $T_{s,max}$ & $T_{s,min}$ & $\overline{T_s}$ & $\overline{T_{s,d}}$ & $\overline{T_{s,n}}$ & $T_{s,max}$ & $T_{s,min}$ \\
Simulation &                  &                      &                      &             &             &                  &                      &                      &             &             \\
\midrule
MassFlux   &            231.8 &                260.0 &                202.7 &       288.3 &       177.1 &            226.6 &                261.7 &                190.5 &       287.3 &       148.7 \\
Adjust     &            249.3 &                268.3 &                229.3 &       294.3 &       214.1 &            241.0 &                272.0 &                209.0 &       299.3 &       166.0 \\
NoCnvPm    &            234.4 &                256.6 &                211.1 &       284.1